In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
print ("Librerías listas, puede continuar.")

Librerías listas, puede continuar.


In [4]:
# Dimensiones
imagen_LR = plt.imread("DIV2K_train_LR_x8/0001x8.png")
imagen_HR = plt.imread("DIV2K_train_HR/0103.png")

alto_LR, ancho_LR, canales_LR = imagen_LR.shape
alto_HR, ancho_HR, canales_HR = imagen_HR.shape

print(f"Las dimensiones de la imagen: {ancho_LR}x{alto_LR}, Número de canales: {canales_LR}")
print(f"Las dimensiones de la imagen: {ancho_HR}x{alto_HR}, Número de canales: {canales_HR}")

Las dimensiones de la imagen: 255x175, Número de canales: 3
Las dimensiones de la imagen: 2040x1140, Número de canales: 3


In [42]:
# Configuración inicial
INPUT_SHAPE = (255, 175, 3)  # Tamaño de entrada para las imágenes de baja resolución
UPSCALE_FACTOR = 2         # Factor de aumento de resolución

# 1. Preprocesamiento de datos
# Cargar y preprocesar el dataset DIV2K
def cargar_y_procesar(data_path):
    """
    Función para cargar imágenes de DIV2K, normalizarlas y preparar pares LR-HR.
    """
    import os
    from tensorflow.keras.preprocessing.image import load_img, img_to_array

    # Directorios donde están las imágenes HR y LR
    lr_dir = os.path.join(data_path, "DIV2K_train_LR_x8")  # Directorio de baja resolución
    hr_dir = os.path.join(data_path, "DIV2K_train_HR")  # Directorio de alta resolución


    low_res_images = []
    high_res_images = []

    # Cargar imágenes de alta resolución y baja resolución
    for lr_img_name in os.listdir(lr_dir):
        # Crear el nombre de la imagen de alta resolución correspondiente
        hr_img_name = lr_img_name.replace("x8.png", ".png")  # Eliminar "x8" para hacer coincidir el formato HR

        # Verifica si la imagen HR correspondiente existe
        if hr_img_name in os.listdir(hr_dir):
            # Carga la imagen de baja resolución (LR)
            lr_img = load_img(os.path.join(lr_dir, lr_img_name), target_size=(255, 175))
            lr_img = img_to_array(lr_img) / 255.0

            # Carga la imagen de alta resolución (HR)
            hr_img = load_img(os.path.join(hr_dir, hr_img_name), target_size=(2040, 1140))
            hr_img = img_to_array(hr_img) / 255.0
        # Convierte las listas a tensores de TensorFlow
        low_res_images = tf.convert_to_tensor(low_res_images)
        high_res_images = tf.convert_to_tensor(high_res_images)

        # Verifica que el número de imágenes coincida
        assert low_res_images.shape[0] == high_res_images.shape[0], "El número de imágenes LR y HR no coincide"

    # Retorna los datos
    return low_res_images, high_res_images

In [ ]:
# Cargamos el conjunto de datos DIV2K
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [43]:
# Carga de datos
data_path = "DIV2K"
lr_images, hr_images = cargar_y_procesar(data_path)

# Dividir en entrenamiento y validación
train_split = int(0.8 * len(lr_images))
x_train, x_val = lr_images[:train_split], lr_images[train_split:]
y_train, y_val = hr_images[:train_split], hr_images[train_split:]

In [44]:
# 2. Construcción del modelo
# Red basada en EfficientNet como extractor de características
def build_sr_model(input_shape):
    """
    Construye un modelo de superresolución basado en EfficientNet y convoluciones.
    """
    # Entrada
    input_layer = layers.Input(shape=input_shape)

    # Bloque EfficientNet como extractor de características
    efficient_net = EfficientNetB0(include_top=False, weights='imagenet', input_shape=input_shape)
    efficient_net.trainable = False  # Congelamos los pesos preentrenados

    x = efficient_net(input_layer)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)

    # Upsampling para aumentar la resolución
    x = layers.Conv2DTranspose(128, (3, 3), strides=(UPSCALE_FACTOR, UPSCALE_FACTOR), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)

    # Reconstrucción final
    output_layer = layers.Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

    # Modelo final
    model = models.Model(inputs=input_layer, outputs=output_layer)
    return model

model = build_sr_model(INPUT_SHAPE)
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 255, 175, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 8, 6, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 6, 256)      │     2,949,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 6, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 8, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 16, 12, 128)    │       295,040 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 16, 12, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 16, 12, 3)      │         3,459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,298,982 (27.84 MB)

 Trainable params: 3,248,643 (12.39 MB)

 Non-trainable params: 4,050,339 (15.45 MB)

In [45]:
# 3. Compilación del modelo
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])

In [47]:
print(x_train.shape)  # Debería ser algo como (n_samples, 255, 175, 3)
print(x_val.shape)    # Debería ser algo como (n_samples, 255, 175, 3)


(0,)
(0,)


In [46]:
# 4. Entrenamiento del modelo
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5),
        tf.keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True)
    ]
)

Epoch 1/50


ValueError: Exception encountered when calling Functional.call().

[1mInvalid input shape for input Tensor("data:0", shape=(16,), dtype=float32). Expected shape (None, 255, 175, 3), but input has incompatible shape (16,)[0m

Arguments received by Functional.call():
  • inputs=tf.Tensor(shape=(16,), dtype=float32)
  • training=True
  • mask=None

In [ ]:
# 5. Evaluación y prueba
# Evaluar en el conjunto de validación
results = model.evaluate(x_val, y_val)
print("Resultados de evaluación:", results)

# Generar ejemplos de predicción
import matplotlib.pyplot as plt

def plot_results(lr_img, sr_img, hr_img):
    """
    Muestra imágenes de baja resolución, superresolución y alta resolución real.
    """
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    titles = ['Low Resolution', 'Super Resolution', 'High Resolution']

    for ax, img, title in zip(axes, [lr_img, sr_img, hr_img], titles):
        ax.imshow(img)
        ax.set_title(title)
        ax.axis('off')

    plt.show()

In [ ]:
# Probar con una imagen
test_lr = x_val[0:1]
test_sr = model.predict(test_lr)
plot_results(test_lr[0], test_sr[0], y_val[0])

# 6. Guardar el modelo final
model.save("sr_model_efficientnet.h5")